In [7]:
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv
import os
import random
import matplotlib.pyplot as plt
import h_louvain_BO as hlbo
import pickle


In [8]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [9]:
def get_AMI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ami = [d[str(i+1)] for i in range(len(HG.nodes))]
    return AMI(gt, A4ami)


In [30]:
## load graph and communities
fn = '/work/home/fcthebe/h-Louvain/h-louvain/datasets/hyperedges-contact-high-school.txt'
with open(fn,"r") as f:
    rd = csv.reader(f)
    lines = list(rd)
Edges = []
for line in lines:
    Edges.append(set(line))

fn = '/work/home/fcthebe/h-Louvain/h-louvain/datasets/node-labels-contact-high-school.txt'
with open(fn, 'r') as file:
    gt = [int(line) for line in file]
gt_dict = {str(i+1):gt[i] for i in range(len(gt))}


In [32]:
HG = hnx.Hypergraph(dict(enumerate(Edges)))
A_gt = [x for x in hmod.dict2part(gt_dict) if len(x)>0]
G = hnx.two_section(HG)
dct = hnx.part2dict(A_gt)
for v in G.vs:
    v['gt'] = dct[v['name']]   


In [39]:
def expo(d, c):
    return 1 if c == d else 0
L = []

for rep in range(30):
    seed = rep
    random.seed(rep)

    hL = hl.hLouvain(HG, hmod_type=expo, delta_it = 0.0001, delta_phase = 0.0001, random_seed = seed) 
    qH_gt = hL.combined_modularity(A_gt, expo, 1)

    ## two-section
    qH_ts = -1
    for ts_rep in range(5):
        ml = G.community_multilevel(weights='weight').membership
        q = hL.combined_modularity(hnx.dict2part(dict(zip(G.vs['name'], ml))), expo, 0)
        if q > qH_ts:
            ami_ts = AMI(G.vs['gt'],ml)
            qH_ts = q 
    V = [ami_ts]
    
    ## strict
    hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
    hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
    df_bo = hBO.hLouvain_perform_BO()
    qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
    ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
    V.append(ami_bo)
    V.append(qH_bo)
    
    ## others
    for ex in [0,1,2,3,4,5,6,7,8]:
        def expo(d, c):
            return (c/d)**(ex/2) if c > d/2 else 0  
        hBO = hlbo.hLouvainBO(HG, hmod_type=expo, delta_it=0.0001, delta_phase=0.0001, random_seed = seed)
        hBO.set_params(hmod_type=expo, show_bo_table=False,seeds=[rep,rep+100,rep+200, rep+300, rep+400])
        df_bo = hBO.hLouvain_perform_BO()
        qH_bo = hL.combined_modularity(df_bo["A"].tolist()[0], expo, 1)
        ami_bo = get_AMI(HG, gt, df_bo["A"].tolist()[0])
        V.append(ami_bo)
        V.append(qH_bo)
    L.append(V)
    print(V)
    fn = 'high-school.pkl'
    with open(fn, 'wb') as fp:
        pickle.dump(L, fp)


[0.958591010653661, 0.8639036770386379, 0.6632655677439184, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.9654583583310026, 0.6210566842394408, 0.9654583583310026, 0.6267354439677114, 0.9654583583310026, 0.6313624245655761, 0.9654583583310027, 0.6351319323206533, 0.9654583583310027, 0.6382024434139475, 0.9267284919548442, 0.6416417223226836, 0.8720266117490332, 0.6455956261034478]
[0.9654583583310027, 0.8720266117490332, 0.6455956261034479, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.9654583583310026, 0.6210566842394408, 0.9654583583310027, 0.6267354439677115, 0.9654583583310026, 0.6313624245655761, 0.9654583583310026, 0.6351319323206533, 0.9654583583310026, 0.6382024434139475, 0.9267284919548442, 0.6416417223226836, 0.8720266117490332, 0.6455956261034479]
[0.9654583583310027, 0.8720266117490332, 0.6455956261034479, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.9654583583310026

[0.9654583583310027, 0.8720266117490332, 0.6455956261034478, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.9654583583310026, 0.6210566842394408, 0.9654583583310026, 0.6267354439677114, 0.9654583583310027, 0.6313624245655761, 0.9654583583310027, 0.6351319323206533, 0.9654583583310027, 0.6382024434139475, 0.9267284919548442, 0.6416417223226836, 0.8720266117490332, 0.6455956261034478]
[0.9654583583310027, 0.8720266117490332, 0.6455956261034479, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.9654583583310026, 0.6210566842394408, 0.9654583583310027, 0.6267354439677115, 0.9654583583310026, 0.6313624245655761, 0.9654583583310026, 0.6351319323206533, 0.9654583583310026, 0.6382024434139475, 0.9267284919548442, 0.6416417223226836, 0.8720266117490332, 0.6455956261034479]
[0.9654583583310027, 0.8720266117490333, 0.6455956261034479, 0.9933016277880199, 0.6060348418989583, 0.9654583583310026, 0.6140879234322923, 0.965458358331002